# 데이터 크롤링

In [19]:
%matplotlib inline
#주피터노트북에서 라인 아래에 보여지도록 설정

%config InlineBackend.figure_formats = {'png','retina'} 
#png 이미지 압축기술설정, retina 그래프 해상도 2배 높히기

------

In [5]:
#get하여 html가져오기

import requests

r = requests.get('https://www.google.co.kr/search?q=KB금융')
r.text[:1000]

'<!doctype html><html itemscope="" itemtype="http://schema.org/SearchResultsPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><link href="/images/branding/product/ico/googleg_lodp.ico" rel="shortcut icon"><title>KB금융 - Google 검색</title><style>#gbar,#guser{font-size:13px;padding-top:1px !important;}#gbar{height:22px}#guser{padding-bottom:7px !important;text-align:right}.gbh,.gbd{border-top:1px solid #c9d7f1;font-size:1px}.gbh{height:0;position:absolute;top:24px;width:100%}@media all{.gb1{height:22px;margin-right:.5em;vertical-align:top}#gbar{float:left}}a.gb1,a.gb4{text-decoration:underline !important}a.gb1,a.gb4{color:#00c !important}.gbi .gb4{color:#dd8e27 !important}.gbf .gb4{color:#900 !important} </style><style>.star{float:left;margin-top:1px;overflow:hidden}.ybhkme{font-size:11px}.j{width:34em}body,td,div,.p,a{font-family:arial,sans-serif;tap-highlig

In [6]:
# 구글 검색결과 갯수 가져오기

import requests
from bs4 import BeautifulSoup
r = requests.get("https://www.google.co.kr/search?q=검색")

soup = BeautifulSoup(r.content, 'lxml')
res_stat = soup.find(id='resultStats').text

print(res_stat)

검색결과 약 17,400,000개


---

In [10]:
#삼성전자 주식정보 가져오기

code = '005930'
url = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd='+code

r = requests.get(url)
soup = BeautifulSoup(r.text,"lxml")
td = soup.find('td', {'class':'cmp-table-cell td0101'})
td

<td class="cmp-table-cell td0101">
<dl>
<dt>
<span class="name">삼성전자</span>
<a class="cEm" href="http://www.samsung.com" target="comPage" title="[홈페이지] www.samsung.com"><img align="absmiddle" hspace="3" src="/images/icon_home.gif"/></a>
<a class="cEm" title="[대표전화] 031-200-1114       
[주식담당] 02-2255-8126"><img align="absmiddle" hspace="1" src="/images/icon_tel.gif"/></a>
<b class="num">005930</b>
</dt>
<dt class="line-left">Samsung Electronics</dt>
<dt class="line-left">KOSPI : 전기전자</dt>
<dt class="line-left">WICS : 반도체와반도체장비</dt>
</dl>
</td>

In [8]:
td.find('span',{"class":'name'}).text

'삼성전자'

In [9]:
anchors = td.find_all('a', {'class':'cEm'})
print(anchors[0]['href'])
print(anchors[1]['title'])

http://www.samsung.com
[주식담당] 02-2255-8126


In [25]:
dts = td.find_all('dt',{"class":"line-left"})
print(dts[0].text)
print(dts[1].text)
print(dts[2].text)

Samsung Electronics
KOSPI : 전기전자
WICS : 반도체와반도체장비


---

In [11]:
# 네이버 실시간 급상승 검색어 크롤링
import re
import requests

html_text= requests.get('http://naver.com').text
re.findall('<span class="ah_k">(.*?)</span>', html_text)[:20]

['아파트투유',
 '배그 점검',
 '포항지진',
 '2018 워터밤',
 '히든싱어',
 '로맨스 패키지 107호',
 '한상규',
 'kt 요금제',
 '헥터',
 '주진우',
 '농협은행인터넷뱅킹',
 '서울시장 후보',
 '국민은행 주택청약',
 '농협',
 '손은서',
 '배틀그라운드',
 '아이수당',
 '슈츠',
 '한국교직원공제회',
 '로맨스 패키지']

In [14]:
# JSON 데이터 받기 - 네이버 실시간 검색어 순위
import json, requests
from pandas.io.json import json_normalize

r = requests.get('http://rank.search.naver.com/rank.js')
json_normalize(json.loads(r.text), ['data','data'])


,change,cvalue,delta,keyword,rank,ratio,score,tvalue
0,+,3450,0,아파트투유,1,.,39,32.13850
1,+,365,0,배그 점검,2,.,267,24.56000
2,+,354,0,포항지진,3,.,345,25.28483
3,+,346,0,2018 워터밤,4,.,549,31.33095
4,+,165,0,히든싱어,5,.,204,19.34039
5,+,190,0,로맨스 패키지 107호,6,.,456,17.98706
6,+,248,1,kt 요금제,7,+,186,14.61466
7,+,122,1,헥터,8,+,633,14.91738
8,+,280,3,서울시장 후보,9,+,129,15.14479
9,+,269,0,주진우,10,.,174,13.36418


In [15]:
# time stamp
r = requests.get('http://rank.search.naver.com/rank.js')
json.loads(r.text)['ts']

'2018-05-31T12:18:00+0900'

---
### 한글인코딩 전화번호부 사이트 
- 한국 사이트중 euc-kr 인코딩이 되어있어 재처리 해야하는 경우

In [17]:
# searchWord:%C8%DE%C0%CF%C1%F8%B7%E1
# x:37.5640907
# y:126.9979403
# dong:
# city:%BC%AD%BF%EF
# gu:%C1%DF%B1%B8
# addr4:
# addr:%BC%AD%BF%EF+%C1%DF%B1%B8

from urllib.parse import quote, unquote

# 이걸 euc-kr인코딩으로 바꿔주면 '휴일진료'라는 글자로 바뀐다.
u = '%C8%DE%C0%CF%C1%F8%B7%E1' 
print (unquote(u, encoding='euc-kr'))

u = '%BC%AD%BF%EF'
print (unquote(u, encoding='euc-kr'))

u = '%BC%AD%BF%EF+%C1%DF%B1%B8'
print (unquote(u, encoding='euc-kr'))

휴일진료
서울
서울+중구


In [18]:
#str(유니코드) -> bytes(EUC-KR) -> URL Quot

b = '휴일진료'.encode('euc-kr')
print( type(b) )
print( quote(b) )

<class 'bytes'>
%C8%DE%C0%CF%C1%F8%B7%E1


In [20]:
# URL Quot (EUC-KR) -> str(유니코드)

u = '%C8%DE%C0%CF%C1%F8%B7%E1'
s = unquote(u, encoding='euc-kr')
print(type(s))
print(s)

<class 'str'>
휴일진료


In [34]:
# 해결책 ------str.encode('euc-kr')--------------
import requests

url = 'http://www.isuperpage.co.kr/search.asp'

data = {
    'searchWord':'휴일진료'.encode('euc-kr'),
    'dong':''.encode('euc-kr'),
    'city':'서울'.encode('euc-kr'),
    'gu':'중구'.encode('euc-kr')
}

r = requests.post(url, data=data)
r.text[:1000]

'\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<title>:: 대한민국 모든 전화번호 검색은 한국전화번호부 ::</title>\r\n<link rel="stylesheet" type=\'text/css\' href="/css3/base_n_n.css"/>\r\n<link href="/css3/dcaccordion.css" rel="stylesheet" type="text/css" />\r\n<link href="/jqy/jquery-ui.css" rel="stylesheet" type="text/css" />\r\n<script type="text/javascript" src="/jqy/jquery-1.11.2.min.js"></script>\r\n<script src="/jqy/jquery-ui-1.10.3.custom.min.js"></script>\r\n<script src="/jqy/jquery.ba-dotimeout.min.js"></script>\r\n<script src="/jqy/common_web.js"></script>\r\n<script src="/jqy/placeholders.min.js"></script>\r\n\r\n  \r\n\r\n\r\n<script language="javascript">\r\n\r\n\r\n$(function() {\r\n\t$("img.lmimg").mouseover(function() {\r\n\t\r\n\t\t$(this).attr("src",$(this).attr("src").replace("off","on"));\r\n\t});\r\n  $("img.lmimg").mouseout(function() {\r\n        $(this).css("display","");\r\n\t\t$(this).at

In [38]:
from bs4 import  BeautifulSoup

soup = BeautifulSoup(r.text, 'lxml')
search_result = soup.find('div', {'id':'search_result'})


In [39]:
lis = search_result.find_all('li')[2:]
for li in lis:
    divs = li.find_all('div')
    
    #div[0]
    title = divs[0].a.text #상호
    spans - divs[0].find_all('span')
    search = spans[1].text #검색어
    
    #div[1]
    spans = divs[1].find_all('span')
    phone = spans[0].text #전화번호
    addr = spans[1].text #주소
    print(spans[2])
    print("%s %s %s %s"%(title, search, phone, addr))   
    

AttributeError: 'NoneType' object has no attribute 'find_all'